In [0]:
zomato_df = spark.sql("SELECT * FROM zomato")

In [0]:

%pip install openpyxl pandas

import pandas as pd
country_wise = pd.read_excel("/Volumes/workspace/default/country_wise/Country-Code.xlsx")
display(country_wise)


In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/country_wise"))
country_wise_spark = spark.createDataFrame(country_wise)
#country_wise_spark.show()
JoinedCountry_df=country_wise_spark.join(zomato_df,'Country Code',"inner")
#JoinedCountry_df.show()

### Inner Join working principle
JoinedCountry_df = zomato_df.join(country_wise_spark, 'Country Code', "inner")
Left DataFrame = zomato_df

The columns of zomato_df appear first in the output.
Then, columns from country_wise_spark (except the join key, which is merged).


In [0]:
JoinedCountry_df.columns

In [0]:
print(len(JoinedCountry_df.columns))
print(len(zomato_df.columns))
print(len(country_wise_spark.columns))

In [0]:
#zomato_df.select("Rating text").show()     # Works
#zomato_df.select('Rating text').show()     # Works
zomato_df.select("Rating color","Restaurant ID").show()

### 🍽️ Zomato Dataset — Analytical SQL Questions

The following are adapted analytical questions based on the **Zomato dataset** (`zomato` table with columns like `Country Code`, `Rating text`, `Rating color`, `Cuisines`, `Latitude`, `Longitude`, `Price range`, etc.).

The queries are written in **Spark SQL** but can also be executed in standard SQL engines with small adjustments.

---

#### Question 01  
**What is the total number of restaurants available in each country?**

```sql
SELECT `Country Code`, COUNT(*) AS total_restaurants
FROM zomato
GROUP BY `Country Code`;
```

#### Question 02  
**How many restaurants fall under each rating category?**

```sql
SELECT `Rating text`, COUNT(*) AS restaurant_count
FROM zomato
GROUP BY `Rating text`
ORDER BY restaurant_count DESC;
```

#### Question 03  
**What are the top 5 cuisines offered globally?**

```sql
SELECT Cuisines, COUNT(*) AS cuisine_count
FROM zomato
GROUP BY Cuisines
ORDER BY cuisine_count DESC
LIMIT 5
```

#### Question 04  
**Which rating color is most common among restaurants?**

```sql
SELECT `Rating color`, COUNT(*) AS rating_count
FROM zomato
GROUP BY `Rating color`
ORDER BY rating_count DESC;
```

#### Question 05  
**For each country, what is the most popular cuisine?**

```sql
SELECT `Country Code`, Cuisines, COUNT(*) AS cuisine_count
FROM zomato
GROUP BY `Country Code`, Cuisines
QUALIFY RANK() OVER(PARTITION BY `Country Code` ORDER BY COUNT(*) DESC) = 1;
```

#### Question 06  
**Which city (latitude/longitude) has the maximum restaurants?**

```sql
SELECT Latitude, Longitude, COUNT(*) AS restaurant_count
FROM zomato
GROUP BY Latitude, Longitude
ORDER BY restaurant_count DESC
LIMIT 1;
```

#### Question 07  
**What is the distribution of restaurants by Price range in each country?**

```sql
SELECT `Country Code`, `Price range`, COUNT(*) AS restaurant_count
FROM zomato
GROUP BY `Country Code`, `Price range`
ORDER BY `Country Code`, `Price range`;
```


#### Question 08  
**How many restaurants in each country accept online delivery?**

```sql
SELECT `Country Code`, COUNT(*) AS online_delivery_restaurants
FROM zomato
WHERE `Has Online delivery` = 'Yes'
GROUP BY `Country Code`;
```

#### Question 09  
**What is the average rating of restaurants per country?**

```sql
SELECT `Country Code`, AVG(`Aggregate rating`) AS avg_rating
FROM zomato
GROUP BY `Country Code`;
```

#### Question 10  
**Create a summary table with Country Code, Cuisines, Rating text, and Price range.**

```sql
SELECT `Country Code`, Cuisines, `Rating text`, `Price range`
FROM zomato;
```

#### Question 11  
**Rank cuisines per country by popularity**

```sql
SELECT `Country Code`, Cuisines, COUNT(*) AS cuisine_count,
       RANK() OVER (PARTITION BY `Country Code` ORDER BY COUNT(*) DESC) AS cuisine_rank
FROM zomato
GROUP BY `Country Code`, Cuisines;
```


Reference Link:- https://www.analyticsvidhya.com/blog/2022/12/case-study-restaurants-insights-using-pyspark-databricks

- Question 01:- What is the total amount each customer spent at the restaurant?
- Question 02:- How many days has each customer visited the restaurant?
- Question 03:- What was each customer’s first item from the menu?
- Question 04:- Find out the most purchased item from the menu and how many times the customers purchased it.
- Question 05:- Which item was the most popular for each customer
- Question 06:- Which item was ordered first by the customer after becoming a restaurant member?
- Question 07:- Which item was purchased before the customer became a member?
- Question 08:- What is the total items and amount spent for each member before they became a member?
- Question 09:- If each rupee spent equates to 10 points and item ‘jeera_rice’ has a 2x points multiplier, find out how many points each customer would have.
- Question 10:- Create the complete table with all data and columns like customer_id, order_date, product_name, price, and member(Y/N).
- Question 11:- We also require further information about the ranking of customer products. The owner does not need the ranking for non-member purchases, so he expects null ranking values for the records when customers still need to be part of the membership program.



In [0]:
result = spark.sql("""
    SELECT `Country Code`, `Rating text`, COUNT(*) as total_restaurants
    FROM zomato
    GROUP BY `Country Code`, `Rating text`
    ORDER BY total_restaurants DESC
""")
result.show()

In [0]:
result=result.withColumnRenamed("Rating text","Rating_text") \
        .withColumnRenamed("Rating text","Rating_text")
result.columns
result.createOrReplaceTempView("zomato_summary")
#result.write.saveAsTable("zomato_summary")